# RDW

In [21]:
import pandas as pd

from visions.core.functional import type_inference_report_frame, compare_detect_inference_frame
from visions.core.implementations import visions_standard_set
from visions.core.implementations.types import *
from visions.core.model.type import evolve_relation
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Gekentekende voertuigen

In [22]:
df = pd.read_csv('../data/rdw/gekentekende_voertuigen.csv', sep=',', nrows=1000)
df.head(10)

,Kenteken,Voertuigsoort,Merk,Handelsbenaming,Vervaldatum APK,Datum tenaamstelling,Bruto BPM,Inrichting,Aantal zitplaatsen,Eerste kleur,Tweede kleur,Aantal cilinders,Cilinderinhoud,Massa ledig voertuig,Toegestane maximum massa voertuig,Massa rijklaar,Maximum massa trekken ongeremd,Maximum trekken massa geremd,Retrofit roetfilter,Zuinigheidslabel,Datum eerste toelating,Datum eerste afgifte Nederland,Wacht op keuren,Catalogusprijs,WAM verzekerd,Maximale constructiesnelheid (brom/snorfiets),Laadvermogen,Oplegger geremd,Aanhangwagen autonoom geremd,Aanhangwagen middenas geremd,Vermogen (brom/snorfiets),Aantal staanplaatsen,Aantal deuren,Aantal wielen,Afstand hart koppeling tot achterzijde voertuig,Afstand voorzijde voertuig tot hart koppeling,Afwijkende maximum snelheid,Lengte,Breedte,Europese voertuigcategorie,Europese voertuigcategorie toevoeging,Europese uitvoeringcategorie toevoeging,Plaats chassisnummer,Technische max. massa voertuig,Type,Type gasinstallatie,Typegoedkeuringsnummer,Variant,Uitvoering,Volgnummer wijziging EU typegoedkeuring,Vermogen massarijklaar,Wielbasis,Export indicator,Openstaande terugroepactie indicator,Vervaldatum tachograaf,Taxi indicator,Maximum massa samenstelling,Aantal rolstoelplaatsen,Maximum ondersteunende snelheid,API Gekentekende_voertuigen_assen,API Gekentekende_voertuigen_brandstof,API Gekentekende_voertuigen_carrosserie,API Gekentekende_voertuigen_carrosserie_specifiek,API Gekentekende_voertuigen_voertuigklasse
0,VH286P,Bedrijfsauto,VOLKSWAGEN,TRANSPORTER,20200326.0,20140326,9605.0,gesloten opbouw,3.0,N.v.t.,N.v.t.,4.0,1968.0,1789.0,2800.0,1889.0,750.0,NaN,Nee,NaN,20140326,20140326,Geen verstrekking in Open Data,39558.0,Ja,NaN,1011.0,NaN,2200.0,2200.0,NaN,NaN,0.0,4.0,0.0,0.0,NaN,489.0,190.0,N1,NaN,NaN,r. schutbord motorruimte,2800.0,7J0,NaN,e1*2007/46*0130*13,GCAAA280X0,NNFM52Z1116NVR27MJG3RS,0.0,0.03,300.0,Nee,Nee,NaN,Nee,4800.0,0.0,NaN,https://opendata.rdw.nl/resource/3huj-srit.json,https://opendata.rdw.nl/resource/8ys7-d773.json,https://opendata.rdw.nl/resource/vezc-m2t6.json,https://opendata.rdw.nl/resource/jhie-znh9.json,https://opendata.rdw.nl/resource/kmfi-hrps.json
1,74VHDJ,Bedrijfsauto,VOLKSWAGEN,GOLF VARIANT VAN D 55 KW,20150904.0,20140926,NaN,gesloten opbouw,NaN,N.v.t.,N.v.t.,4.0,1896.0,1177.0,1750.0,1277.0,500.0,1200.0,Nee,NaN,19990602,19990602,Geen verstrekking in Open Data,NaN,Nee,NaN,573.0,NaN,NaN,NaN,NaN,NaN,0.0,4.0,0.0,0.0,NaN,0.0,170.0,N1,NaN,NaN,op schutbord onder motorkap,1750.0,NaN,NaN,NaN,NaN,NaN,0.0,0.04,248.0,Nee,Nee,NaN,Nee,2950.0,NaN,NaN,https://opendata.rdw.nl/resource/3huj-srit.json,https://opendata.rdw.nl/resource/8ys7-d773.json,https://opendata.rdw.nl/resource/vezc-m2t6.json,https://opendata.rdw.nl/resource/jhie-znh9.json,https://opendata.rdw.nl/resource/kmfi-hrps.json
2,GJ685D,Personenauto,CITROEN,CITROEN C1,20210529.0,20151021,1081.0,hatchback,4.0,GRIJS,Niet geregistreerd,3.0,998.0,815.0,1240.0,915.0,NaN,NaN,NaN,A,20150529,20150529,Geen verstrekking in Open Data,14425.0,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,0.0,0.0,NaN,347.0,0.0,M1,NaN,NaN,r. op bodemplaat by voorzitting,1240.0,P*****,NaN,e11*2001/116*0238*11,S,CFB2-H1B000,0.0,0.06,234.0,Nee,Nee,NaN,Nee,1240.0,0.0,0.0,https://opendata.rdw.nl/resource/3huj-srit.json,https://opendata.rdw.nl/resource/8ys7-d773.json,https://opendata.rdw.nl/resource/vezc-m2t6.json,https://opendata.rdw.nl/resource/jhie-znh9.json,https://opendata.rdw.nl/resource/kmfi-hrps.json
3,V303GS,Bedrijfsauto,MERCEDES-BENZ,CITAN,20210616.0,20181019,5645.0,gesloten opbouw,2.0,N.v.t.,N.v.t.,4.0,1197.0,1344.0,1810.0,1444.0,NaN,NaN,NaN,NaN,20170616,20170616,Geen verstrekking in Open Data,27883.0,Ja,NaN,466.0,NaN,NaN,NaN,NaN,NaN,0.0,4.0,0.0,0.0,NaN,432.0,183.0,N1,NaN,NaN,in motorruimte r.,1810.0,X,NaN,e2*2007/46*0130*18,FXA4,FXA4J3,0.0,0.06,270.0,Nee,Nee,NaN,Nee,0.0,0.0,0.0,https://opendata.rdw.nl/resource/3huj-srit.json,https://opendata.rdw.nl/resource/8ys7-d773.json,https://opendata.rdw.nl/resource/vezc-m2t6.json,https://opendata.rdw.nl/resource/jh

In [23]:
df.dtypes

Kenteken                                              object
Voertuigsoort                                         object
Merk                                                  object
Handelsbenaming                                       object
Vervaldatum APK                                      float64
Datum tenaamstelling                                   int64
Bruto BPM                                            float64
Inrichting                                            object
Aantal zitplaatsen                                   float64
Eerste kleur                                          object
Tweede kleur                                          object
Aantal cilinders                                     float64
Cilinderinhoud                                       float64
Massa ledig voertuig                                 float64
Toegestane maximum massa voertuig                    float64
Massa rijklaar                                       float64
Maximum massa trekken on

Type problems:

* (1) Columns starting with `Aantal` are Counts/Unsigned Integers
* (2) Urls are not recognized
* (3) Nullable integers are represented as Float
* (4) String columns are stored as objects
* (5) Dutch booleans (Ja/Nee) are represented as objects
* (6) Integer dates (e.g. 20200808) are not recognized
* (7) The order of "Zuinigheidslabel" is lost (A,B,C,D,E,F)

In [24]:
typeset = visions_standard_set()

report = type_inference_report_frame(df, typeset)
print(report)

Maximum ondersteunende snelheid                        visions_float                  != visions_bool                   
Cilinderinhoud                                         visions_float                  != visions_integer                
Plaats chassisnummer                                   visions_string                 == visions_string                 
Typegoedkeuringsnummer                                 visions_string                 == visions_string                 
Vermogen massarijklaar                                 visions_float                  == visions_float                  
Merk                                                   visions_string                 == visions_string                 
Openstaande terugroepactie indicator                   visions_string                 == visions_string                 
Afstand hart koppeling tot achterzijde voertuig        visions_float                  != visions_integer                
Handelsbenaming                 

The standard typeset mitigates the problems (3) and (4).

We can resolve problems (1) and (2) by including `visions_count` and `visions_url` in the typeset.

`visions_bool` recognizes some strings by default (e.g. Y/N). We evolve the type to include Dutch coercions to resolve (5).

To resolve (6) we evolve the `visions_datetime` type with an integer relation.

Finally, we include an evolved version of `visions_ordinal` to the typeset that has an inference relation from strings (A,B,C, etc.).

In [5]:
from visions.lib.relations.string_to_ordinal import string_to_ordinal
from visions.lib.relations.string_to_bool import get_boolean_coercions
from visions.lib.relations.integer_to_datetime import integer_to_datetime_year_month_day


typeset = visions_standard_set()

# (1)
typeset += visions_count

# (2)
typeset += visions_url

# (5)
typeset = typeset.replace(
    visions_bool, 
    visions_bool.make_string_coercion('nl', get_boolean_coercions('nl'))
)

# (6)
typeset = typeset.replace(
    visions_datetime,
    evolve_relation(visions_datetime, 'str_yyyymmdd', integer_to_datetime_year_month_day)
)

# (7)
typeset += evolve_relation(visions_ordinal, 'str_alpha', string_to_ordinal)

report = type_inference_report_frame(df, typeset)
print(report)

Maximum ondersteunende snelheid                        visions_float                  == visions_float                  
Cilinderinhoud                                         visions_float                  != visions_integer                
Plaats chassisnummer                                   visions_string                 == visions_string                 
Typegoedkeuringsnummer                                 visions_string                 == visions_string                 
Vermogen massarijklaar                                 visions_float                  == visions_float                  
Merk                                                   visions_string                 == visions_string                 
Openstaande terugroepactie indicator                   visions_string                 == visions_string                 
Afstand hart koppeling tot achterzijde voertuig        visions_float                  != visions_integer                
Handelsbenaming                 

## Erkende bedrijven

In [25]:
df = pd.read_csv('../data/rdw/erkende_bedrijven.csv', sep=',')
df.head(10)

,Volgnummer,Naam bedrijf,Gevelnaam,Straat,Huisnummer,Huisnummer toevoeging,Postcode numeriek,Postcode alfanumeriek,Plaats,api_bedrijf_erkenningen
0,7195995,QUALITY DESIGN & SERVICES,QUALITY DESIGN & SERVICES,KROMMEDIJK,15,NaN,7381,BT,KLARENBEEK,https://opendata.rdw.nl/resource/nmwb-dqkz.json
1,476301,PIEST B.V.,NaN,OLIESLAGWEG,43,NaN,7521,HX,ENSCHEDE,https://opendata.rdw.nl/resource/nmwb-dqkz.json
2,8536694,M&O Investment B.V.,Royal Cars & Watches,ROSBAYERWEG,153,NaN,1521,RW,WORMERVEER,https://opendata.rdw.nl/resource/nmwb-dqkz.json
3,8580416,Autoschade Herstel Midden Nederland B.V.,Autoschade Herstel Midden Nederland B.V.,DE SMALLE ZIJDE,44,NaN,3903,LP,VEENENDAAL,https://opendata.rdw.nl/resource/nmwb-dqkz.json
4,8484102,LOUWMAN DEALERBEDRIJVEN B.V.,Louwman Amsterdam West,PELERIN,1,NaN,1046,CC,AMSTERDAM,https://opendata.rdw.nl/resource/nmwb-dqkz.json
5,7010412,DE GROOT AUTOMOTIVE,De Groot Automotive,WATTSTRAAT,7,B - C,2723,PZ,ZOETERMEER,https://opendata.rdw.nl/resource/nmwb-dqkz.json
6,7108166,ADR TAXI,Autoschade de Roeck,TAMBOERIJNHOF,15,NaN,1544,ZA,ZAANDIJK,https://opendata.rdw.nl/resource/nmwb-dqkz.json
7,8285739,BUTZELAAR AUTO'S,BUTZELAAR AUTO'S,HARDERWIJKERWEG,128,B,3852,AG,ERMELO,https://opendata.rdw.nl/resource/nmwb-dqkz.json
8,7762200,HANDELSONDERNEMING SANPA V.O.F.,Handelsonderneming Sanpa,MARCONISTRAAT,17,NaN,4461,HH,GOES,https://opendata.rdw.nl/resource/nmwb-dqkz.json
9,1179161,WIELIES,WIELIES,MIJLSTRAAT,31,NaN,5281,LJ,BOXTEL,https://opendata.rdw.nl/resource/nmwb-dqkz.json


In [26]:
df.dtypes

Volgnummer                  int64
Naam bedrijf               object
Gevelnaam                  object
Straat                     object
Huisnummer                  int64
Huisnummer toevoeging      object
Postcode numeriek           int64
Postcode alfanumeriek      object
Plaats                     object
api_bedrijf_erkenningen    object
dtype: object

Type problems:

* (1) Volgnummer, Huisnummer and Postcode are positive numbers
* (2) `api_bedrijf_erkenningen` should be recognized as URL
* (3) String columns are stored as objects

In [8]:
typeset = visions_standard_set()

report = type_inference_report_frame(df, typeset)
print(report)

Huisnummer                   visions_integer                == visions_integer                
Plaats                       visions_string                 == visions_string                 
Postcode alfanumeriek        visions_string                 == visions_string                 
Volgnummer                   visions_integer                == visions_integer                
Straat                       visions_string                 == visions_string                 
Huisnummer toevoeging        visions_string                 == visions_string                 
Postcode numeriek            visions_integer                == visions_integer                
Gevelnaam                    visions_string                 == visions_string                 
api_bedrijf_erkenningen      visions_string                 == visions_string                 
Naam bedrijf                 visions_string                 == visions_string                 
In total 0 out of 10 types were changed.



(1) is mitigated by including the `visions_count` type in the typeset. By default it does not coerce positive integers, hence we evolve it with the `integer_to_count` relation.

(2) is mitigated by including the `visions_url` type

(3) is mitigated by default in the standard typeset

In [27]:
from visions.lib.relations.integer_to_count import integer_to_count

# (1)
typeset += evolve_relation(visions_count, 'uint', integer_to_count)

# (2)
typeset += visions_url

report = type_inference_report_frame(df, typeset)
print(report)

Huisnummer                   visions_integer                != visions_count[uint]            
Plaats                       visions_string                 == visions_string                 
Postcode alfanumeriek        visions_string                 == visions_string                 
Volgnummer                   visions_integer                != visions_count[uint]            
Straat                       visions_string                 == visions_string                 
Huisnummer toevoeging        visions_string                 == visions_string                 
Postcode numeriek            visions_integer                != visions_count[uint]            
Gevelnaam                    visions_string                 == visions_string                 
api_bedrijf_erkenningen      visions_string                 != visions_url                    
Naam bedrijf                 visions_string                 == visions_string                 
In total 4 out of 10 types were changed.



## Parkeren

In [28]:
df = pd.read_csv('../data/rdw/parkeren_gebied.csv', sep=',')
df.head(10)

,AreaManagerId,AreaId,AreaDesc,StartDateArea,EndDateArea,UsageId
0,2448,603_BOGP4,In de Bogaard P4 (Rijswijk),20140101,29991231,GARAGEP
1,303,303_STRAND,Strand Drontermeer,20170101,29991231,BETAALDP
2,2448,518_GRMA,Garage Grote Markt (Den Haag),20131125,29991231,GARAGEP
3,1900,1900_LEMM,Lemmerweg,20140101,29991231,BETAALDP
4,796,020121,Luybenstraat,20110930,20111001,BETAALDP
5,281,CWAA,Waalkade,20171101,29991231,VERGUNP
6,361,TESTGEBIED,Gebied zonder kernwinkeltijden,20160101,20170101,ONTHEFFING
7,164,3004,Parkeerterrein Eikstraat,20120210,20140101,BETAALDP
8,385,BZVOLENDAM,Blauwe zone Volendam,20120605,29991231,BLAUWEZ
9,344,V10200,Nieuw Welgelegen west,20131201,29991231,VERGUNP


In [11]:
df.dtypes

AreaManagerId     int64
AreaId           object
AreaDesc         object
StartDateArea     int64
EndDateArea       int64
UsageId          object
dtype: object

Type problems:

* (1) The RDW stores dates (StartDateArea and EndDateArea) as integers. Missing values are encoded as `29991231`.
* (2) String columns are stored as objects
* (3) Positive numbers can be represented as counts.

In [12]:
typeset = visions_standard_set()

report = type_inference_report_frame(df, typeset)
print(report)

EndDateArea        visions_integer                == visions_integer                
AreaDesc           visions_string                 == visions_string                 
StartDateArea      visions_integer                == visions_integer                
UsageId            visions_string                 == visions_string                 
AreaManagerId      visions_integer                == visions_integer                
AreaId             visions_string                 == visions_string                 
In total 0 out of 6 types were changed.



The problems are mitigated as before, except that we now replace the NaN value with None.

In [31]:
from visions.lib.relations.integer_to_datetime import integer_to_datetime_year_month_day
from visions.lib.relations.integer_to_count import integer_to_count

# TODO: Create a transformer for custom NaNs
df.loc[df.EndDateArea==29991231, 'EndDateArea'] = None

typeset = typeset.replace(
    visions_datetime,
    evolve_relation(visions_datetime, 'yyyymmdd', integer_to_datetime_year_month_day)
)

typeset += evolve_relation(visions_count, 'uint', integer_to_count)

# Final result
report = type_inference_report_frame(df, typeset)
print(report)

EndDateArea        visions_float                  != visions_datetime[yyyymmdd]     
AreaDesc           visions_string                 == visions_string                 
StartDateArea      visions_integer                != visions_datetime[yyyymmdd]     
UsageId            visions_string                 == visions_string                 
AreaManagerId      visions_integer                != visions_count[uint]            
AreaId             visions_string                 == visions_string                 
In total 3 out of 6 types were changed.

